In [2]:
import requests
import pandas as pd
import numpy as np
import pygsheets
import datetime
from datetime import date
import json
import webbrowser
from time import sleep
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Importing functions
import sys
sys.path.append(r'C:\Users\Usuario\Desktop\Learning-Testing\PyStrava\Scripts')
from Functions import *

In [3]:
# Specify the path to your JSON file
file_path = r'C:\Users\Usuario\Desktop\Learning-Testing\PyStrava\Outputs\test_all_workouts.json'

# Open the JSON file and load its contents into a Python object
with open(file_path, 'r') as json_file:
    all_workouts_json = json.load(json_file)

In [4]:
# This function will parse the workout json, grab the laps portion and clean the units
def CleanLapsJson(workout_json):
    # parsing json
    df = pd.json_normalize(workout_json, 'laps')
    df = df[['activity.id',
            'id',
            'lap_index',
            'moving_time',
            'distance',
            'average_speed',
            'max_speed',
            'split',
            'start_index',
            'end_index',
            'total_elevation_gain',
            'average_cadence',
            'average_heartrate',
            'max_heartrate',
            'pace_zone']]

    # renaming laps id
    df = df.rename(columns={'activity.id' : 'activity_id',
                            'id' : 'lap_id',
                            'moving_time' : 'lap_time_min',
                            'distance' : 'lap_distance',
                            'average_heartrate' : 'lap_avg_heartrate',
                            'max_heartrate' : 'lap_max_heartrate_km/h',
                            'average_speed' : 'lap_avg_speed_km/h',
                            'max_speed' : 'lap_max_speed_km/h',
                            'total_elevation_gain' : 'lap_total_elevation_gain',
                            'average_cadence' : 'avg_cadence'})

    # Formatting lap distance to km
    df['lap_distance'] = round(df['lap_distance']/1000,2)

    # Formatting lap time to minutes
    df['lap_time_min'] = round(df['lap_time_min']/60,2)
    df['lap_time_min'] = round(df['lap_time_min'] - (df['lap_time_min']%1) + (((df['lap_time_min'] % 1) * 60) / 100),2)

    # Formatting lap start index to minutes
    df['start_index'] = round(df['start_index']/60,2)
    df['start_index'] = round(df['start_index'] - (df['start_index']%1) + (((df['start_index'] % 1) * 60) / 100),2)
    
    # Formatting lap end index to minutes
    df['end_index'] = round(df['end_index']/60,2)
    df['end_index'] = round(df['end_index'] - (df['end_index']%1) + (((df['end_index'] % 1) * 60) / 100),2)   
    
    # Formatting speed to km/h
    df['lap_avg_speed_km/h'] = df['lap_avg_speed_km/h'] * 3.6
    df['lap_max_speed_km/h'] = df['lap_max_speed_km/h'] * 3.6
    
    return df

In [5]:
laps_df = CleanLapsJson(all_workouts_json)
laps_df.to_csv(r'C:\Users\Usuario\Desktop\Learning-Testing\PyStrava\Outputs\laps_table.csv')

In [6]:
laps_df

,activity_id,lap_id,lap_index,lap_time_min,lap_distance,lap_avg_speed_km/h,lap_max_speed_km/h,split,start_index,end_index,lap_total_elevation_gain,avg_cadence,lap_avg_heartrate,lap_max_heartrate_km/h,pace_zone
0,10793103601,37409418720,1,7.06,0.0,0.000,0.0000,1,0.00,3.58,0.0,NaN,123.2,138.0,NaN
1,10793103601,37409418723,2,6.38,0.0,0.000,0.0000,2,3.59,7.24,0.0,NaN,135.2,164.0,NaN
2,10793103601,37409418724,3,6.01,0.0,0.000,0.0000,3,7.25,11.16,0.0,NaN,142.3,179.0,NaN
3,10793103601,37409418725,4,1.28,0.0,0.000,0.0000,4,11.17,12.08,0.0,NaN,122.1,153.0,NaN
4,10783927933,37370586045,1,5.49,0.0,0.000,0.0000,1,0.00,3.26,0.0,NaN,116.7,140.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,9359950371,31929436462,5,3.53,0.0,0.000,0.0000,5,10.56,12.18,0.0,NaN,101.2,151.0,NaN
912,9347431801,31885166871,1,7.25,1.0,8.100,11.1456,1,0.00,1.16,2.8,85.2,135.3,145.0,0.0
913,9347431801,31885166872,2,7.54,1.0,7.596,10.3788,2,1.17,2.31,0.0,85.0,142.0,149.0,0.0
914,9347431801,31885166874,3,7.33,1.0,7.956,10.3284,3,2.32,3.51,3.6,84.1,147.8,156.0,0.0
